## SOLID Principles

What are the SOLID Principles?

SOLID Principles was mentioned in a paper in 2000 by Robert C. Martin who is famous for his book on clean code and is also known as Uncle Bob. The objective of this principle is to reduce problem from occuring in the first place. Distinguishing between style and substance. It is an aid and not a rigid formula since an alternative to a class will always exist and the term best is moot as it depends on the non functional requirements. It is focused on the ripple effect of from change and how the software fits its intended use. It is a set of principles listed below:

S - Single Responsibility
O - Open/Closed Principles
L - Liskov Substitution
I - Interface Segregation
D - Dependency Inversion

Understanding these principles enable us to write better codes that are reusable and extensible. It also enables us to perform code reviews that are not just stylistic by articulating why it is not designed well and what and why it should be improved on.

Other known principles such as <font color='red'>`SOLID`</font> are `DRY` (Do Not Repeat Yourself) and `GRASP` (General Responsibility Assignment Software Principle). To ensure that the principle is practiced properly, static checkers such as mypy and tests needs to be implemented for every classes or functions. Without proper tests in place, refactoring and writing code based on the SOLID principles will be very difficult.

Note:
-----
This notebook will comprise of some theory on the principle and some practical examples of how we can refactor code to achieve the desired state of the principle

Single Responsibility
----

We want the classes to have a single responsibility or high in cohesion. But what is the meaning of a single responsibility? This can be confusing and we can use principles such as GRASP to determine the extant of a single responsibility for a class. For the sake of simplicity, we want to ensure that the class has a single purpose that is consistent with the definition of the class which allows for easier reusability.

For example, we have an Order class. This class should allow users to add or remove items, check the status of the payment and the total amount that needs to be paid. However, it should not be responsible for performing the action of paying for the order as well. This means that, we are splitting the responsibility of the class into its logical purposes.


In [4]:
class Order:
    """A simple Order Class which allows user to list the name of the products, its quantity and its price"""

    def __init__(self):
        self.items = []
        self.quantities = []
        self.prices = []
        self.status = "open"

    def add_item(self, name: str, quantity: int, price: float) -> None:
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)

    def total_price(self) -> float:
        total = 0
        for i in range(len(self.prices)):
            total += self.quantities[i] * self.prices[i]
        return total


class PaymentProcessor:
    """Class that process the payment method"""

    def pay_debit(self, order: str, security_code: str) -> None:
        print("Processing debit payment type")
        print(f"Verifying security code: {security_code}")
        order.status = "paid"

    def pay_credit(self, order: str, security_code: str) -> None:
        print("Processing credit payment type")
        print(f"Verifying security code: {security_code}")
        order.status = "paid"


order = Order()
order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)
print(order.total_price())
print(order.items)

payment_method = PaymentProcessor()
payment_method.pay_credit(order, "1234")

210
['Keyboard', 'SSD', 'USB cable']
Processing credit payment type
Verifying security code: 1234


Open/ Closed Principle
-------------

A software entity should be open for extension but closed for modification. This means that we should extend the existing code to enable more functionality but avoid modifying existing code in order to achieve that additional feature. 

Lets update the PaymentProcessor class to satisfy the Open/ Closed Principle. We could first create an abstract class, and then we can extend the payment method such as debit, credit, stripe, bitcoin ... by creating new subclasses

In [5]:
from abc import ABC, abstractmethod


class PaymentProcessor(ABC):
    """A Base payment processor class with the required interface for payment"""

    @abstractmethod
    def pay(self, order: str, security_code: str) -> None:
        """This method needs to be implemented for each subclass"""
        pass


class DebitPaymentProcessor(ABC):
    """A Debit payment processor class"""

    def pay(self, order: str, security_code: str) -> None:
        print("Processing debit payment type")
        print(f"Verifying security code: {security_code}")
        order.status = "paid"


class CreditPaymentProcessor(ABC):
    """A Credit payment processor class"""

    def pay(self, order: str, security_code: str) -> None:
        print("Processing credit payment type")
        print(f"Verifying security code: {security_code}")
        order.status = "paid"


class PaypalPaymentProcessor(ABC):
    """A Paypal payment processor class"""

    def pay(self, order: str, security_code: str) -> None:
        print("Processing bitcoin payment type")
        print(f"Verifying security code: {security_code}")
        order.status = "paid"


payment_method = DebitPaymentProcessor()
payment_method.pay(order, "1234")

Processing debit payment type
Verifying security code: 1234


Liskov Substitution
-----

The idea of this principle is that, if you were to have an object in a program, you could replace that object with an instance of its subclass where the properties of its superclass is not altered (able to retain the correctness of the program). In other words, objects of a superclass can be replaced by its subclass. 

In our example, paypal is using security code to perform the processing of the payment but in reality, it requires email address to perform its functionality. In essence, we are abusing the security code attribute. This violates the Liskov Substitution rule as passing in email as a security code does not retain the correctness of the program although python will allow for it. We have several methods to resolve this. 

1) Move the attribute to the initialization of the subclass

In [9]:
from abc import ABC, abstractmethod


class PaymentProcessor(ABC):
    """A Base payment processor class with the required interface for payment"""

    @abstractmethod
    def pay(self, order: str, security_code: str) -> None:
        """This method needs to be implemented for each subclass"""
        pass


class DebitPaymentProcessor(ABC):
    """A Debit payment processor class"""

    def __init__(self, security_code: str) -> None:
        self.security_code = security_code

    def pay(self, order: str) -> None:
        print("Processing debit payment type")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


class CreditPaymentProcessor(ABC):
    """A Credit payment processor class"""

    def __init__(self, security_code: str) -> None:
        self.security_code = security_code

    def pay(self, order: str) -> None:
        print("Processing credit payment type")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


class PaypalPaymentProcessor(ABC):
    """A Paypal payment processor class"""

    def __init__(self, email_address: str) -> None:
        self.email_address = email_address

    def pay(self, order: str) -> None:
        print("Processing bitcoin payment type")
        print(f"Verifying email address: {self.email_address}")
        order.status = "paid"


payment_method = PaypalPaymentProcessor("sample@gmail.com")
payment_method.pay(order)

Processing bitcoin payment type
Verifying email address: sample@gmail.com


Interface Segregation
-----

It is better to have several interfaces instead of a single generic interface for a superclass. This would mean that, if the subclass will not be implementing that particular interface (method), there is no need to have it in the superclass.

To think of it in another way, we need to know what does the collaborator class need? We need to decompose the interface into key features. We can use the verb to identify what should be decomposed. The goal is to isolate the ripple of changes. If were to add another reader or loader class, it should not have an impact on the analysis class or other application.

In this example, we will see how we can extend the functionality of the super class to another base class that requires authentication mechanism and build subclasses from them. Note, how the other payment processor class does not require sms authentication but the classes that do require it, will be inheriting from the new abstract base class.

We could also utilize composition, where instead of creating multiple abstract subclasses, we allow the subclass of an abstract class to be composed of functionalities that allows for that functionality

In [13]:
# Interface Segregation using Subclassing/ Inheritance

from abc import ABC, abstractmethod


class PermissionError(Exception):
    """Exception when users are not allowed to perform the payment action as they are not verified"""


class PaymentProcessor(ABC):
    """A Base payment processor class with the required interface for payment"""

    @abstractmethod
    def pay(self, order: str, security_code: str) -> None:
        """This method needs to be implemented for each subclass"""
        pass


class SMSPaymentProcessor(PaymentProcessor):
    """A Base payment processor class with an SMS Authentication interface"""

    @abstractmethod
    def sms_authenticate(self, code: str) -> None:
        """This method needs to be implemented for each subclass"""
        pass


class DebitPaymentProcessor(SMSPaymentProcessor):
    """A Debit payment processor class"""

    def __init__(self, security_code: str) -> None:
        self.security_code = security_code
        self.verified = False

    def sms_authenticate(self, code: str) -> None:
        print("Authenticating using SMS Code")
        print(f"Verifying security code: {code}")
        self.verified = True

    def pay(self, order: str) -> None:
        if not self.verified:
            raise PermissionError("Not allowed to pay")
        print("Processing debit payment type")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


class CreditPaymentProcessor(ABC):
    """A Credit payment processor class"""

    def __init__(self, security_code: str) -> None:
        self.security_code = security_code

    def pay(self, order: str) -> None:
        print("Processing credit payment type")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


payment_processor = DebitPaymentProcessor("1234")
payment_processor.sms_authenticate("9999")
payment_processor.pay(order)

Authenticating using SMS Code
Verifying security code: 9999
Processing debit payment type
Verifying security code: 1234


In [18]:
# Interface Segregation using Composition

from abc import ABC, abstractmethod


class PermissionError(Exception):
    """Exception when users are not allowed to perform the payment action as they are not verified"""


class PaymentProcessor(ABC):
    """A Base payment processor class with the required interface for payment"""

    @abstractmethod
    def pay(self, order: str, security_code: str) -> None:
        """This method needs to be implemented for each subclass"""
        pass


class SMSAuth:
    """Simple Authorization class"""

    authorized = False

    def verify_code(self, code: str) -> None:
        print(f"Verifying code: {code}")
        self.authorized = True

    def is_authorized(self) -> bool:
        return self.authorized


class DebitPaymentProcessor(PaymentProcessor):
    """A Debit payment processor class"""

    def __init__(self, security_code: str, authorizer: SMSAuth) -> None:
        self.security_code = security_code
        self.authorizer = authorizer

    def pay(self, order: str) -> None:
        if not self.authorizer.is_authorized:
            raise PermissionError("Not allowed to pay")
        print("Processing debit payment type")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


sms_authorizer = SMSAuth()
payment_processor = DebitPaymentProcessor("1234", sms_authorizer)
sms_authorizer.verify_code("9999")
payment_processor.pay(order)

Verifying code: 9999
Processing debit payment type
Verifying security code: 1234


Dependency Inversion
-----

Classes should depend on abstraction and not on concrete subclasses. There is a saying that:

A. High-level modules should not depend on low-level modules. Both should depend on abstractions. 
B. Abstractions should not depend upon details. Details should depend upon abstractions

Dependency inversion reduces coupling. It utilizes the concept called Late Binding where it defers making connection with other components until it is running.

For our current use case, we can perform dependency inversion by creating an abstract class for the SMS Authorizer Class. Once that is done, we can also create additional authorizer with that has the same abstract class


In [20]:
# Interface Segregation using Composition

from abc import ABC, abstractmethod


class PermissionError(Exception):
    """Exception when users are not allowed to perform the payment action as they are not verified"""


class PaymentProcessor(ABC):
    """A Base payment processor class with the required interface for payment"""

    @abstractmethod
    def pay(self, order: str, security_code: str) -> None:
        """This method needs to be implemented for each subclass"""
        pass


class Authorizer(ABC):
    """Authorization Base class"""

    @abstractmethod
    def is_authorized(self) -> bool:
        return self.authorized


class SMSAuth(Authorizer):
    """Simple Authorization class"""

    authorized = False

    def verify_code(self, code: str) -> None:
        print(f"Verifying code: {code}")
        self.authorized = True

    def is_authorized(self) -> bool:
        return self.authorized


class NotARobotAuth(Authorizer):
    """Simple Not a Robot Authorization class"""

    authorized = False

    def not_a_robot_verifier(self) -> None:
        print(f"Click the square that has the bicycle")
        self.authorized = True

    def is_authorized(self) -> bool:
        return self.authorized


class DebitPaymentProcessor(PaymentProcessor):
    """A Debit payment processor class"""

    def __init__(self, security_code: str, authorizer: Authorizer) -> None:
        self.security_code = security_code
        self.authorizer = authorizer

    def pay(self, order: str) -> None:
        if not self.authorizer.is_authorized:
            raise PermissionError("Not allowed to pay")
        print("Processing debit payment type")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


sms_authorizer = NotARobotAuth()
payment_processor = DebitPaymentProcessor("1234", sms_authorizer)
sms_authorizer.not_a_robot_verifier()
payment_processor.pay(order)

Click the square that has the bicycle
Processing debit payment type
Verifying security code: 1234


In [22]:
# Another example of Dependency Inversion
# Lets say that you want to create an application that reads from another source
from abc import ABC, abstractmethod


class DataSource(ABC):
    """A Base DataSource class with the required interface for payment"""

    @abstractmethod
    def get_data(self) -> str:
        """This method needs to be implemented for each subclass"""
        pass


class Database(DataSource):
    """Extract Data from Database"""

    def get_data(self) -> str:
        return "Getting data from Database"


class API(DataSource):
    """Extract Data from an API"""

    def get_data(self) -> str:
        return "Getting data from API"


class FrontEnd:

    def __init__(self, data_source: DataSource):
        self.data_source = data_source

    def display_data(self) -> None:
        data = self.data_source.get_data()
        print(data)


database_obj = Database()
front_end_1 = FrontEnd(database_obj)
front_end_1.display_data()

api_obj = API()
front_end_2 = FrontEnd(api_obj)
front_end_2.display_data()

Getting data from Database
Getting data from API


References
----
1) [Solid Principles in Python](https://www.youtube.com/watch?v=pTB30aXS77U)
2) [Composition vs Inheritance](https://www.youtube.com/watch?v=0mcP8ZpUR38)
3) [GRASP Design Principle](https://www.youtube.com/watch?v=fGNF6wuD-fg&t=165s)
4) [Solid Principle Programming Principle](https://www.linkedin.com/learning/learning-solid-programming-principles/reading-and-building-samples?autoSkip=true&resume=false&u=68077770)
5) [Solid principle Real Python](https://realpython.com/solid-principles-python/)

